In [1]:
import pandas as pd
import time
import sys
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, KFold
import multiprocessing
import os
from tqdm import tqdm
from scipy import sparse

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from sklearn import ensemble

In [3]:
import datetime as dt
from datetime import date
import calendar
from calendar import weekday, day_name

In [4]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy as sp

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
from patsy import dmatrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy as sp
from sklearn.datasets import make_regression

In [6]:
all_data_result = pd.read_csv('another_all_train_matrix.csv', sep=',')

In [ ]:
# col = ['tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
#        'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
#        'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
#        'avgspeed']
# all_data_result = all_data_result.drop(col, axis=1)
# all_data_result

In [7]:
weather_file = pd.read_csv("tae/weather_processed.csv", sep = ',')

In [8]:
test = pd.read_csv('tae/test.csv', parse_dates=['date'])
weather = pd.read_csv('tae/weather.csv', parse_dates=['date'])
key = pd.read_csv('tae/key.csv')
test_key = pd.merge(test, key)
test_merge =  pd.merge(test_key, weather, on=['date', 'station_nbr'], how='left')
test_merge.tail()

,date,store_nbr,item_nbr,staion_nbr,station_nbr,tmax,tmin,tavg,depart,dewpoint,...,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
526912,2014-08-02,40,107,NaN,13,81,51,66,M,47,...,-,-,,M,0.00,24.44,30.12,5.7,17,6.5
526913,2014-08-02,40,108,NaN,13,81,51,66,M,47,...,-,-,,M,0.00,24.44,30.12,5.7,17,6.5
526914,2014-08-02,40,109,NaN,13,81,51,66,M,47,...,-,-,,M,0.00,24.44,30.12,5.7,17,6.5
526915,2014-08-02,40,110,NaN,13,81,51,66,M,47,...,-,-,,M,0.00,24.44,30.12,5.7,17,6.5
526916,2014-08-02,40,111,NaN,13,81,51,66,M,47,...,-,-,,M,0.00,24.44,30.12,5.7,17,6.5


In [10]:
def storage(result, store) :
    """
    회귀분석 결과 모음

    전역변수로 아래 데이터 프레임 선언
    result_sets = pd.DataFrame(columns = ["R square", "AIC", "BIC", "P_fstatics", "P_omnibus", "P_jb", "Cond_no"])

    Variables_name.fit() 상태로 삽입
    ex)result = model.fit() 이면
    result를 삽입    


    reason = 이전 데이터 셋에서 무엇을 변화시켰는지 적을 것.
    """    
    global result_sets
    result_sets = pd.DataFrame(columns = ["store_nbr", "R square", "F_statics", "Prob(F_statics)", "AIC", "BIC",
                                          "Omnibus", "P_omnibus", "P_jb", "Cond_no"])
                
    model = sm.OLS.from_formula("units ~ C(item_nbr) + year + "
                                 "C(month) + weekday + is_weekend + is_holiday_weekday + "
                                 "is_holiday_weekend + C(around_BlackFriday) + "
                                 "tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + "
                                 "sunset + snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir"
                                 "+ avgspeed", data=result)
    result = model.fit()

    summary = result.summary()
    
    t1 = summary.tables[0]
    t2 = summary.tables[1]
    t3 = summary.tables[2]
    
    r_squared = float(t1.data[0][3])
    fstatics = float(t1.data[2][3])
    p_fstatics = float(t1.data[3][3])
    AIC = float(t1.data[5][3])
    BIC = float(t1.data[6][3])
    

    omnibus = float(t3.data[0][3])
    p_omnibus = float(t3.data[1][1])
    p_jb = float(t3.data[2][3])
    con = float(t3.data[3][3])

    put = {
        "store_nbr" : store,
        "R square" : r_squared,
        "F_statics" : fstatics,
        "Prob(F_statics)" : p_fstatics,
        "AIC" : AIC,
        "BIC" : BIC,
        "Omnibus" : omnibus,
        "P_omnibus" : p_omnibus,
        "P_jb" : p_jb,
        "Cond_no" : con,  
    }
    
    result_sets.loc[len(result_sets)] = put
    
    for i in range(2, len(t2)):
        result_sets = result_sets.append({
            "Name" : t2.data[i][0],
            "std err" : t2.data[i][2],
            "t" : t2.data[i][3],
            "P>|t|" : t2.data[i][4],
            "0.025" : t2.data[i][5],
            "0.975" : t2.data[i][6],
        }, ignore_index=True)
        
        
    # 모델별 p-value가 10% 이하인 것만 추출
    # 추출된 모델을 토대로 predict




    return result_sets

In [11]:
columns_list = list(all_data_result.columns)


for i in columns_list:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-11-d2bbaf74c646>, line 5)

In [13]:
ls_store_nbr = set(all_data_result.store_nbr)
mod = sys.modules[__name__]
df = pd.DataFrame()
for store_nbr in tqdm(ls_store_nbr):
    data = all_data_result[all_data_result['store_nbr'] == store_nbr]
    mergedata = pd.merge(data, weather_file, on = ['date'])
    tmp_df = storage(mergedata, store_nbr)
    df = df.append(tmp_df, ignore_index=True)

  0%|          | 0/45 [00:00<?, ?it/s]


PatsyError: Error evaluating factor: NameError: name 'wetbulb' is not defined
    units ~ C(item_nbr) + year + C(month) + weekday + is_weekend + is_holiday_weekday + is_holiday_weekend + C(around_BlackFriday) + tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir+ avgspeed
                                                                                                                                                                              ^^^^^^^

In [17]:
df.to_csv("model_OLS.csv", sep=',', index=None)

In [24]:
model2 = sm.OLS.from_formula("units ~ store_nbr + item_nbr + station_nbr_x + day + year + "
                                 "month + weekday + is_weekend + is_holiday + is_holiday_weekday + "
                                 "is_holiday_weekend + holiday_name + around_BlackFriday + station_nbr_y + "
                                 "tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + "
                                 "sunset + snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir"
                                 "+ avgspeed", data=store_1_data)
result_model2 = model2.fit()
result_model2.summary()

NameError: name 'store_1_data' is not defined

In [23]:
def get_features(data, start_num=0, end_num=None, scale=False):
    features = list(data.columns)[start_num:end_num]
    feature_n = len(features)

    if scale:
        features = list(map(lambda x: "scale({})".format(x), features))
        features = " + ".join(features)

    else:
        features = " + ".join(features)

    return features

a = str(get_features(store_1_data, 0))
a

NameError: name 'store_1_data' is not defined

In [22]:
for store_nbr in ls_store_nbr:
    data = all_data_result[all_data_result['store_nbr'] == store_nbr]
    model2 = sm.OLS.from_formula("units ~ weekday + month + is_holiday_weekend", data=data)
    result_model2 = model2.fit()
    print(result_model2.summary())

NameError: name 'ls_store_nbr' is not defined